# PGFinder Interactive Notebook

This notebook is a basic user interface to allow researchers less familiar with the command line to run PGFinder. Some compromises have been made to make it run as a Jupyter notebook on the free myBinder platform.

To use the code to analyse your data you must work from top to bottom on this notebook, following instructions as you go. The first step is to execute all of the "cells" in the notebook so they are ready for your input. 

To do this, **click *Kernel*>*Restart & Run All* on the menu, above.**

In [ ]:
import base64
import codecs
import io
import os
import ipywidgets as widgets
from ipywidgets import HTML
from IPython.display import display
from ipysheet import from_dataframe
import pandas as pd

from pgfinder import matching, validation

# Get list of modifications
allowed_mods = validation.allowed_modifications()

# Get built in mass lists
mass_lists_path = './data/masses'
mass_lists = os.listdir(mass_lists_path)
mass_lists.append('Upload Custom')

# Utility functions
def safe_load_deconvoluted():
    try:
        file_name = list(data_uploader.value.keys())[0]
        file_contents = data_uploader.value[list(data_uploader.value.keys())[0]]['content'] # I hate this line of code
        if not file_name.find('ftrs') == -1:
            return matching.ftrs_reader(io.BytesIO(file_contents))
        elif not file_name.find('txt') == -1:
            return matching.maxquant_file_reader(io.BytesIO(file_contents))
        else:
            raise ValueError('Unknown file type.')
    except:
        print("Could not upload and display file, please make sure you have selected a file of the correct type.")

def safe_load_masses():
    try:
        file_contents = mass_uploader.value[list(mass_uploader.value.keys())[0]]['content'] # I hate this line of code
        return matching.theo_masses_reader(io.BytesIO(file_contents))
    except:
        print("Could not upload and display mass library, please make sure you have selected a file of the correct type.")
        
# Main analysis function
def analysis(b):
    # Upload maxquant file
    uploaded_df = safe_load_deconvoluted()
    
    # Load mass list
    if rb_masses.value == 'Upload Custom':
        theo_masses = safe_load_masses()
    else:
        csv_filepath = os.path.join(mass_lists_path, rb_masses.value)
        theo_masses = matching.theo_masses_reader(csv_filepath)
    
    # Make sure mod list is a list
    mod_list = list(selector_mods.value)
    
    # Validate inputs
    validation.validate_raw_data_df(uploaded_df)
    validation.validate_theo_masses_df(theo_masses)
    validation.validate_enabled_mod_list(mod_list)

    results = matching.data_analysis(uploaded_df, theo_masses, 0.5, mod_list, 10)
    
    # Make the download button
    results_csv_str = results.to_csv()
    filename = 'results.csv'
    b64 = base64.b64encode(results_csv_str.encode())
    payload = b64.decode()

    html_buttons = '''<html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <a download="{filename}" href="data:text/csv;base64,{payload}" download>
    <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
    </a>
    </body>
    </html>
    '''

    html_button = html_buttons.format(payload=payload,filename=filename)
    display(HTML(html_button))
    
# Define widgets

# Layout for a bigger button
big_button = widgets.Layout(width='auto')

# Deconvoluted Data file upload
data_uploader = widgets.FileUpload(
    accept = '.txt,.ftrs', 
    description = 'Upload Deconvoluted Data',
    multiple = False,
    layout = big_button
) 

# Modifcation selector
selector_mods = widgets.SelectMultiple(
    options = allowed_mods,
    description = 'Modification',
    disabled = False
)

# Mass library selector
rb_masses = widgets.RadioButtons(
    options = mass_lists,
    description = 'Mass List',
    disabled = False
)

# Mass library file upload
mass_uploader = widgets.FileUpload(
    accept = '.csv', 
    description = 'Upload Mass Library',
    multiple = False,
    layout = big_button
) 

# Analysis button
button = widgets.Button(description="Run Analysis")
button.on_click(analysis)

## Step 1: Upload Deconvoluted Data
Click *Upload* to upload a `.txt` file output by MaxQuant ([example file](https://github.com/Mesnage-Org/pgfinder/raw/master/data/maxquant_test_data.txt)), or an `.ftrs` file.

In [ ]:
display(data_uploader)

## Step 2: Select Modifications
Select modifications (Hold down control / command and click to select mulitple items.)

In [ ]:
display(selector_mods) 

## Step 3: Select or Upload Mass Library

### Select

In [ ]:
display(rb_masses)

### (Optional) Upload Custom Mass Library
[Example mass library file.](https://raw.githubusercontent.com/Mesnage-Org/pgfinder/master/data/masses/e_coli_monomer_masses.csv)

In [ ]:
display(mass_uploader)

## Step 4: Run Analysis
Click run analysis.
After the analysis is complete, a download button will appear.

In [ ]:
display(button)